In [13]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np
import os

MNIST_PATH = os.path.join(os.getcwd(),'dataset','mnist')
MNIST_FILE = os.path.join(MNIST_PATH,'mnist.npz')

# load dataset from tensorflow
(x_train, y_train), (x_test, y_test) = mnist.load_data(path=MNIST_FILE)
x_train , x_test = x_train/255.0 , x_test/255.0

classes = 10

np.savez_compressed(os.path.join(MNIST_PATH, 'x_test'), x_test)
np.savez_compressed(os.path.join(MNIST_PATH, 'y_test'), y_test)
np.savez_compressed(os.path.join(MNIST_PATH, 'x_norm'), x_train[::10])

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\itznu\\PycharmProjects\\use-of-spiking-nn\\snn-toolbox-examples\\dataset'

In [3]:
nn_model = tf.keras.Sequential([
    layers.Flatten(input_shape=[28, 28]),
    layers.Dense(1024, activation='relu'),
    layers.Dense(2048, activation='relu'),
    layers.Dense(classes, activation='softmax')
])

nn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])

nn_model.fit(x_train, y_train, epochs=10, batch_size=512)

Epoch 1/10
118/118 [==============================] - 8s 68ms/step - loss: 13.7642 - accuracy: 0.8328

In [8]:
import pathlib

test_loss,test_acc = nn_model.evaluate(x_test,y_test)
print('Accuracy:',test_acc) # cca 96.67%

313/313 [==============================] - 2s 6ms/step - loss: 0.2462 - accuracy: 0.9599

In [ ]:
# pip install -q pyyaml h5py
model_path = os.getcwd() + '\\model\\'
if not os.path.exists(model_path):
    os.makedirs(model_path)


nn_model_path = pathlib.Path('model\\mnist-nn')
nn_model.save(model_path + 'mnist_ann_model.h5')

In [15]:
# https://github.com/NeuromorphicProcessorProject/snn_toolbox/blob/master/examples/mnist_keras_INI.py
from snntoolbox.bin.run import main
from snntoolbox.utils.utils import import_configparser

dt = .1
time_steps_ms = 30
batch_size = 512
test_samples = 10000

config_parser = import_configparser()
config = config_parser.ConfigParser()

config['paths'] = {
    'path-wd': os.getcwd() + '\\model\\',
    'dataset_path': MNIST_PATH,
    'filename_ann': os.getcwd() + '\\model\\mnist_ann_model'
}

config['tools'] = {
    'evaluate_ann': True,
    'normalize': True
}

config['simulation'] = {
    'simulator': 'INI',
    'duration': time_steps_ms,
    'num_to_test': test_samples,
    'batch_size': batch_size,
    'dt': dt,
    'keras_backed': 'tensorflow'
}

config['cell'] = {
    'tau_refrac': dt,
    'delay': dt
}

config['output'] = {
    'plot_vars': {                  # Various plots (slows down simulation).
        'spiketrains',              # Leave section empty to turn off plots.
        'spikerates',
        'activations',
        'correlation',
        'v_mem',
        'error_t'}
}

path = os.getcwd() + '\\config\\'
if not os.path.exists(path):
    os.makedirs(path)

# this for some reason doesnt create the directory
config_filepath = os.getcwd() + '\\config\\mnist-snn-config'
with open(config_filepath, 'w+') as config_file:
    config.write(config_file)


main(config_filepath)

RuntimeWarning: Data set file 'x_test.npz' or 'y_test.npz' was not found in specified data set path C:\Users\itznu\PycharmProjects\use-of-spiking-nn\snn-toolbox-examples\dataset.